# Python Document Search Engine

Now, we will start to put together all of the topics that we have studied so far into a series of "Python Recipes"---coding examples that illustrate the power of thinking hard about how data is organized and structured. In the first example, we will consider a "Python Search Engine" that will identify relevant items given a query string.

We're going to start with a dataset of tweets about airlines:

In [1]:
import csv

def load_data(filename):
    rtn = []
    #open the file with the csv reader
    with open(filename, newline='') as csvfile:
        tweets = csv.reader(csvfile, delimiter=',', quotechar='"') 
        next(tweets)#skip the header   
        for row in tweets:
            rtn.append(row[10])
    return rtn

tweets = load_data('Tweets.csv')

#figure out how much data we have
size = sum([i.__sizeof__() for i in tweets]) + tweets.__sizeof__()

print('Number: ', len(tweets), '\t Size:', size/1e6,'MB','\t Bytes per tweet:', size/len(tweets))

Number:  14640 	 Size: 2.416432 MB 	 Bytes per tweet: 165.05683060109288


This dataset contains a large list of tweets represented as string. We want to be able to search for phrases in these tweets. Of course, the first thing that we can do is the simple naive search routine where we scan through the entire dataset.

## Naive Search
Suppose, we wanted to find a substring in this collection of tweets, we could write the following code that iterates through each tweet and searches for a substring:

In [2]:
import datetime

def find(phrase, tweets):
    #Naive full scan approach
    start = datetime.datetime.now()
    rtn = []    
    for t in tweets:
        if phrase in t:
            rtn.append(t)
    print('Find() elapsed time: ', (datetime.datetime.now()-start).total_seconds())          
    return rtn


find('choppy landing', tweets)

find('LAX', tweets)

Find() elapsed time:  0.002149
Find() elapsed time:  0.002356


["@VirginAmerica So excited for my first cross country flight LAX to MCO I've heard nothing but great things about Virgin America. #29DaysToGo",
 '@VirginAmerica LAX to EWR - Middle seat on a red eye. Such a noob maneuver. #sendambien #andchexmix',
 '@VirginAmerica help, left expensive headphones on flight 89 IAD to LAX today. Seat 2A. No one answering L&amp;F number at LAX!',
 '@VirginAmerica plz help me win my bid upgrade for my flight 2/27 LAX---&gt;SEA!!!  🍷👍💺✈️',
 '@VirginAmerica just landed in LAX, an hour after I should of been here. Your no Late Flight bag check is not business travel friendly #nomorevirgin',
 '@VirginAmerica trying to add my boy Prince to my ressie. SF this Thursday @VirginAmerica from LAX http://t.co/GsB2J3c4gM',
 '@VirginAmerica Can you find us a flt out of LAX that is sooner than midnight on Monday? That would be great customer service 😃',
 '@VirginAmerica congrats, you just got all my business from EWR to SFO/LAX. Fuck you @united fl1289 SFO/EWR was the cl

In [3]:
find("Landed at JFK", tweets)

Find() elapsed time:  0.002151


[]

That's pretty fast (5  ms!) But imagine if you had to run a million of such lookups, that would be 5000 seconds! At scale, small overheads add up. 

Now, we use our "inverted indexing" trick to make such searches faster.

## Inverted Index
Next, we will try to do the same search with an inverted index. The indexing structure that we will use is a python dictionary.

In [4]:
import string 

def build_index(tweets):
    start = datetime.datetime.now() 
    index = {}
    
    #some code to deal with punctuation
    table = str.maketrans('', '', string.punctuation)
    for i,t in enumerate(tweets):      
        words = t.translate(table).split()       
        for w in words:         
            if w not in index:
                index[w] = set()          
            index[w].add(i) #add a pointer to the relevant tweet       
    print('build_index() elapsed time: ', (datetime.datetime.now()-start).total_seconds())      
    return index

index = build_index(tweets)

build_index() elapsed time:  0.261896


Notice that build_index is about a 100x slower than a single query. What does this mean? Basically, indexing is only valuable if you run a lot of queries! 

The next challenge is how to use an inverted index to answer general substring queries. In class, we showed how to do exact keyword lookup but the phrase 'choppy landing' is actually two words. This is actually not a problem, and we can use the inverted index to retrieve a set of candidates and then use the naive find method among just those candidates.

So, let's write a new find function that can use this index:
* It splits the phrase into its constituent words
* Searches each word in the inverted index, finds a set of possibly relevant tweets (that match on a single word)
* Then double checks that set.

In [5]:
len(index["landed"]), len(index["Chicago"]), len(index["in"])


(73, 73, 2245)

In [6]:
def find_index(phrase, tweets, index):
    start = datetime.datetime.now()
    words = phrase.split()
    #find tweets that contain all words
    candidates = None
    
    for w in words: #for each words in the phrase
        try:
            if candidates is None:
                candidates = index[w] #return the set of tweets for w
            else:
                candidates = candidates.intersection(index[w])
        except KeyError:
            return []
    
    candidate_tweets = [tweets[ref] for ref in candidates]
    return find(phrase, candidate_tweets)
    print('find_index() elapsed time: ', (datetime.datetime.now()-start).total_seconds())
    
find_index('choppy landing', tweets, index)
find_index('LAX', tweets, index)[:10]

Find() elapsed time:  3e-06
Find() elapsed time:  2.6e-05


["@united is doing musicians real dirty at LAX. I've never been blocked from getting on a flight with my bass.",
 "@United I'm hoping we don't miss our LAX - ITO connection. Not looking forward to being stuck at LAX overnight with our team....AGAIN!",
 "@united I forgot that Intl flights out of LAX don't go from Intl Terminal! Easiest re-check in ever! woo!",
 "@united - you sure missed the mark on tonight's redeye from LAX to Chicago. What a mess! You can do better!",
 "@VirginAmerica So excited for my first cross country flight LAX to MCO I've heard nothing but great things about Virgin America. #29DaysToGo",
 '@VirginAmerica LAX to EWR - Middle seat on a red eye. Such a noob maneuver. #sendambien #andchexmix',
 '@VirginAmerica help, left expensive headphones on flight 89 IAD to LAX today. Seat 2A. No one answering L&amp;F number at LAX!',
 "@USAirways I have been doing that all day. Can't find my bag anywhere bc they're saying it was never scanned &amp; technically never left LAX.",

In essence, you are paying a small upfront cost for greatly improved find performance (nearly a 1000x faster!). Speed is only aspect of search engine performance. We also like to support situations where a user mistypes a phrase. For example, if we mistype choppy landing:

In [7]:
find_index('chopy landing', tweets, index)

[]

In [8]:
find_index('choppy landing', tweets, index)





Find() elapsed time:  3e-06


['@VirginAmerica pilot says we expect a choppy landing in NYC due to some gusty winds w/a temperature of about 5 degrees &amp; w/the windchill -8']

In [9]:
len(index.keys())

20042

Our system returns nothing. Can we write a fast suggestion utility that can quickly identify typos.

## Did you mean? 
So now we are going to write a utility that can identify mispelling and typos and suggest potential alternatives. So let's start off with a naive approach that simply finds the closest word in the index in terms of edit distance:

In [ ]:
#! pip install Distance

In [10]:
import distance

print( "Jaccard('a b', 'b c')=", distance.jaccard('a b', 'b c'))
print( "Levenshtein('a b', 'b c')=", distance.levenshtein('a b','b c') )

Jaccard('a b', 'b c')= 0.5
Levenshtein('a b', 'b c')= 2


In [11]:
def did_you_mean_naive(word, index):
    start = datetime.datetime.now()
    if word in index:
        return word
    else:
        distances = [(distance.levenshtein(word, iw), iw) for iw in index]
        distances.sort()
        print('did_you_mean_naive() elapsed time: ', (datetime.datetime.now()-start).total_seconds())
        return distances[0][1]
    
    

did_you_mean_naive('chopy', index)

did_you_mean_naive() elapsed time:  0.482608


'choppy'

In [12]:
did_you_mean_naive("word", index)

'word'

In [25]:
did_you_mean_naive("virginamericA", index)

did_you_mean_naive() elapsed time:  1.014001


'virginamerica'

In [13]:
did_you_mean_naive('discont', index)

did_you_mean_naive() elapsed time:  0.582688


'discount'

In [14]:
did_you_mean_naive('arival', index)

did_you_mean_naive() elapsed time:  0.553647


'arrival'

In [15]:
len(index.keys())

20042

The suggestion utility runs much slower than the actual query!!! How do we fix this? We can use the same trick as before: a fast algorithm to find reasonable candidates and a slower algorithm to refine those candidates.

In fact, we will use an inverted index again. Just this time over sub-sequences of letters and not words. The first thing that we are going to do is to calculate n-grams these are contiguous sub-sequences of letters.

In [16]:
#ngram
#def find_ngrams(word, n):
#    return list(zip(*[word[i:] for i in range(n)]))

def find_ngrams(word, n):
    '''digest a word (a string) into a list of len(word)-n+1 
    ngrams of length n.'''
    return [word[i:i+n] for i in range(0, len(word)-n+1)]

find_ngrams('dave', 2)

['da', 'av', 've']

Next, we are going to build a "word" index, an indexing structure that maps ngrams to words that contain them.

In [17]:
def build_word_index(index, n):
    '''Builds a dictionary that maps ngrams contained in the 
    keys of index to the keys themselves.'''
    start = datetime.datetime.now()  
    word_index = {}
    for word in index:
        ngrams = find_ngrams(word, n)     
        for subseq in ngrams:       
            if subseq not in word_index:
                word_index[subseq] = set()       
            word_index[subseq].add(word) #add a pointer to the relevant word
    
    print('build_word_index() elapsed time: ', (datetime.datetime.now()-start).total_seconds())
    return word_index

word_index = build_word_index(index, 3)
print(repr(word_index)[0:400])


build_word_index() elapsed time:  0.113169
{'Vir': {'VirginAmerica', '“VirginAmericaYouve', '😂VirginAmerica', 'VirginAtlantic', '“VirginAmerica', 'Virginia', 'wantVirginAmerica', 'Virgins', 'WeRVirgin', 'Virgin', 'Virtual'}, 'irg': {'diehardvirgin', '😂VirginAmerica', 'VirginAtlantic', '“VirginAmerica', 'plannedneverflyvirginforbusiness', 'Virginia', 'virginAmerica', 'SouthwestAirgive', '“VirginAmericaYouve', 'virginmedia', 'Virgins', 'WeRV


In [18]:
list(word_index.keys())[0:10]

['Vir', 'irg', 'rgi', 'gin', 'inA', 'nAm', 'Ame', 'mer', 'eri', 'ric']

In [19]:
len(word_index.keys()) 

23728

In [23]:
hist = { k: len(word_index[k]) for k in word_index.keys()}
hist

{'Vir': 11,
 'irg': 23,
 'rgi': 38,
 'gin': 94,
 'inA': 11,
 'nAm': 9,
 'Ame': 30,
 'mer': 165,
 'eri': 148,
 'ric': 105,
 'ica': 163,
 'Wha': 8,
 'hat': 76,
 'dhe': 3,
 'hep': 5,
 'epb': 1,
 'pbu': 1,
 'bur': 30,
 'urn': 34,
 'sai': 44,
 'aid': 17,
 'plu': 7,
 'lus': 15,
 'you': 71,
 'ouv': 4,
 'uve': 5,
 'add': 26,
 'dde': 15,
 'ded': 80,
 'com': 235,
 'omm': 63,
 'mme': 31,
 'erc': 29,
 'rci': 11,
 'cia': 56,
 'ial': 52,
 'als': 44,
 'the': 222,
 'exp': 57,
 'xpe': 39,
 'per': 120,
 'rie': 97,
 'ien': 87,
 'enc': 98,
 'nce': 167,
 'tac': 28,
 'ack': 112,
 'cky': 10,
 'did': 10,
 'idn': 12,
 'dnt': 16,
 'tod': 20,
 'oda': 28,
 'day': 97,
 'Mus': 3,
 'ust': 170,
 'mea': 19,
 'ean': 49,
 'nee': 26,
 'eed': 51,
 'tak': 20,
 'ake': 64,
 'ano': 13,
 'not': 88,
 'oth': 84,
 'her': 179,
 'tri': 70,
 'rip': 30,
 'its': 45,
 'rea': 178,
 'eal': 62,
 'all': 248,
 'lly': 115,
 'agg': 30,
 'ggr': 6,
 'gre': 41,
 'res': 233,
 'ess': 238,
 'ssi': 100,
 'siv': 21,
 'ive': 180,
 'bla': 18,
 'las': 6

We can use this word index to build a more sophisticated search:
* Only consider words that share a minimum number of ngrams with the lookup word

In [21]:
def did_you_mean_better(word, word_index, n, thresh=1):
    '''Finds the closest key in index to the query word, but only check
    for words that share at least one ngram with the query word.  Uses
    word_index.
    '''
    start = datetime.datetime.now()
    
    candidate_words = {}
    ngrams = find_ngrams(word, n)
    
    for ngram in ngrams:
        candidates = word_index.get(ngram, set())
        
        for candidate in candidates:
            candidate_words[candidate] = candidate_words.get(candidate,0) + 1
    
    
    
    distances = [(distance.levenshtein(word, iw), iw) for iw in candidate_words if candidate_words[iw] >= thresh]
    distances.sort()
        
    print('did_you_mean_better() elapsed time: ', (datetime.datetime.now()-start).total_seconds())
        
    return distances[0][1]
    

did_you_mean_better('chopy', word_index, 3)

did_you_mean_better() elapsed time:  0.001953


'choppy'

Notice how much faster this approach is!! 0.992237 secs v.s. 0.003581 seconds.

## Putting it all together

Now, let's write the full program and try out some queries

In [26]:
def find_final(phrase, \
               tweets, \
               index, \
               word_index, \
               n=3, \
               thresh=1):
    print('Searching for...' + phrase + " in " + str(len(tweets)) + " tweets")
    out = find_index(phrase, tweets, index)
    print('Found ' + str(len(out)) + ' matches')
    
    if len(out) == 0:
        for word in phrase.split():
            if word not in index:
                print('Did you mean: ' + did_you_mean_better(word, word_index, n, thresh) + ' instead of ' + word + '?')
    else:
        print(out)

find_final('choppy landing', tweets, index, word_index)

Searching for...choppy landing in 14640 tweets
Find() elapsed time:  5e-06
Found 1 matches
['@VirginAmerica pilot says we expect a choppy landing in NYC due to some gusty winds w/a temperature of about 5 degrees &amp; w/the windchill -8']


In [27]:
find_final('chopy landing', tweets, index, word_index)

Searching for...chopy landing in 14640 tweets
Found 0 matches
did_you_mean_better() elapsed time:  0.001831
Did you mean: choppy instead of chopy?


In [28]:
find_final('choppy landig', tweets, index, word_index)

Searching for...choppy landig in 14640 tweets
Found 0 matches
did_you_mean_better() elapsed time:  0.021358
Did you mean: landing instead of landig?


In [ ]:
find_final('LAX', tweets, index, word_index)

In [ ]:
find_final('LAXS', tweets, index, word_index)